<a target="_blank" href="https://colab.research.google.com/github/apiwat-chantawibul/anon-workshop/blob/main/notebooks/examples/examples.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

ถ้ารันบน Colab ต้อง download file ข้อมูลตั้งต้นก่อน ด้วยการรัน cell ด้านล่าง:

In [1]:
!test -f source.csv || curl -OLf https://github.com/apiwat-chantawibul/anon-workshop/raw/refs/heads/main/notebooks/examples/source.csv

# Examples of Anonymized Datasets

- เราจะเริ่มจากตัวอย่างข้อมูลตั้งต้นชุดเดียว
- นำมาแสดงผลลัพธ์ของการทำ anonymization 3 รูปแบบหลักๆ คือ
  - Publish ใน[รูปแบบตาราง](#Anonymized-Dataset-as-Tables)
  - Publish ใน[รูปแบบสถิติ](#Anonymized-Dataset-as-Summary-Statistics)
  - Publish ใน[รูปแบบ API](#Anonymized-Dataset-as-API) ให้เรียกถามเป็นครั้งๆไป
- เพื่อให้มีภาพเป้าหมายอยู่ในใจ และเห็นความเป็นไปได้หลายๆแบบก่อน
- จากนั้นเราค่อยมาเรียนรู้ข้อดี, ข้อเสีย, และวิธีการทำ anonymization แต่ละรูปแบบ

## ข้อมูลตั้งต้น

- เป็นข้อมูลจำลอง
  - สร้างมาจาก [generate_mock_source.ipynb](generate_mock_source.ipynb)
  - แล้วเก็บไว้เป็น file [source.csv](source.csv)
- โปรแกรมด้านล่าง
  - อ่านชุดข้อมูล [source.csv](source.csv) มาใส่ในตัวแปรชื่อ `original`
  - จากนั้นแสดงส่วนหนึ่งของตารางข้อมูลออกมา

In [2]:
import pandas as pd

original = pd.read_csv(
    'source.csv',
    parse_dates = ['date_of_birth'],
    dtype = {'postcode': str},
)
original

,passport,first_name,last_name,job,date_of_birth,comment,postcode,gender,spending
0,694369456,ปัตถพงษ์,ถนัดรบ,ศิลปิน,1963-03-09,กำไรซ่อนโน่นห่วงใยและทั่วชาว ฤดูคนตายหมดคู่ทุ่...,75320,Female,10541
1,229512554,เมษา,ทันยุค,เจ้าหน้าที่รัฐบาล,1998-01-30,น้ำเย็นกำเทเลแกรมฝาก ตะปูตีนอุปโภคโดยหยอกครูขี...,21410,Male,13130
2,U96857223,พัชรพร,ตระกูลบุญ,ผู้จัดพิมพ์,2021-12-31,ดูฝากร่มรื่นถางหน้ากาก คำสั่งรากแต่แม่พยาธิ \n...,75320,Male,11953
3,H26958060,ประจิน,ทวีเดช,นักสังคมศาสตร์,1943-01-20,สีแข็งอัศจรรย์มิตรรสชาติดึกหมัด \nดีแพ้คำถามสถ...,95840,Female,12285
4,144271331,กิติวัฒน์,บุญญาไลย์,ศิลปิน,1960-11-25,กิ่งไม้ไข่พับกว้าง ยาเขยื้อนสำหรับแกสดชื่นโรงส...,95840,Male,10399
...,...,...,...,...,...,...,...,...,...
995,D76609557,โกมล,ดำริห์ชอบ,นักการศึกษา,2020-06-05,เย็นขนมตรวจเลี้ยวองค์น้าแรก เศษอินสตาแกรมกู ทำ...,95840,Female,8251
996,R93321926,นิมุ,นักสำหรวจ,บรรณารักษ์,1913-01-31,พิพักพิพ่วนสหภาพนักเรียนความกิ่งไม้ ขุดแรกโก๋แ...,88530,Male,12172
997,134186584,สุพัตร,เนื้อนุ่ม,นักสังคมสงเคราะห์,2021-10-26,วิ่งสุดท้ายหนุนข้างแพ้รอปู คบมะเขือเทศพับถางลู...,57110,Female,9902
998,T25822912,จินต์จุฑา,ธรรมนิยม,นักสังคมสงเคราะห์,1973-10-12,บันไดขันน้ำผ่อวิชาชีพผลิตขี้ดินขาดเคิ่งย่อ ตอก...,57110,Female,10554


### ลักษณะข้อมูลตั้งต้น

- สมมุติว่าเป็นข้อมูลจำนวนเงินที่สมาชิกในโครงการหนึ่งๆเบิกใช้จ่ายไป และความเห็นของสมาชิกประกอบการใช้จ่ายนั้นๆ
- มี attribute ที่ระบุถึงตัวคนเจ้าของข้อมูลโดยตรง (**direct identfier**) อยู่สามอย่าง คือ
  - passport
  - first_name
  - last_name
- มี atttribute ที่ระบุตัวตนได้ทางอ้อม (**indirect identifier**) เพราะมีความเป็นไปได้สูงว่าจะประกฎในชุดข้อมูลอื่น คือ
  - date_of_birth
  - postcode
  - gender
- มี attribute อื่นๆที่ไม่น่าจะพบได้ในแหล่งข้อมูลอื่น เนื่องจากเป็นข้อมูลเฉพาะกับ คือ
  - spending --- คือ จำนวนเงินที่ได้ใช้จ่ายไป
  - comment --- คือ ความเห็นของสมาชิกประกอบการใช้จ่าย

## Anonymized Dataset as Tables

- ด้านล่างคือโปรแกรมผลิตข้อมูลนิรนามจากชุดข้อมูลตั้งต้นและผลลัพธ์ในรูปแบบตาราง
  - click "..." เพื่อแสดงโปรแกรมที่ถูกซ่อนอยู่
- ขั้นตอนที่ใช้ประกอบไปด้วยหลายเทคนิค แต่ละขั้นทำให้ข้อมูลแม่นยำน้อยลง แต่ปกปิดตัวตนของสมาชิกผู้เบิกค่าใช้จ่ายและ comment มากขึ้น ตามมาตรวัดความนิรนาม [K-anonymity](../k_anonymity/k_anonymity.ipynb)
- โดยเฉพาะขั้นตอนสุดท้าย ซึ่งตัดรายการที่โดนเด่นเกินไปไม่มีพวกออกทั้งหมด ทำให้ข้อมูลลดลงจาก 1000 เหลือเพียง 125 รายการ แต่ทำให้รับประกันได้ว่าผ่านระดับความนิรนาม 3-anonymity
- รายละเอียดการทำงานแต่ละขั้นจะถูกอธิบายโดยละเอียดในบท [ฺBasic Anonymization Techniques](../techniques/basic.ipynb)

In [3]:
# remove direct identifiers
table = original.drop(columns = ['passport', 'first_name', 'last_name'])

# convert to age group
from datetime import datetime
import numpy as np
table['age'] = datetime.now().year - table['date_of_birth'].dt.year
table['age'] = pd.cut(
    table['age'],
    bins = [0, 10, 20, 30, 40, 50, 60, np.inf],
    right = False,
)
table = table.drop(columns = 'date_of_birth')

# Round spending to nearest 1000
table['spending'] = table['spending'].round(-3)

# Mask lower postcode digits
table['postcode'] = table['postcode'].str[:-3] + 'xxx'

# Sort into groups by indirect identifiers
table = table.set_index(['gender', 'age', 'postcode', 'job']).sort_index()

# NOTE: See Proportion of records in each group first should aid decision:
#     table.index.value_counts().plot.hist()

# NOTE: In pandas 2.2.3, there is a bug that causes "FutureWarning: The default of observed=False is deprecated ..." when calling `index.value_counts`.
# This has been fixed in developing version, but not yet released at the time of writing.
# Meanwhile, use the following to ignore said warning:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Suppress records belonging to small groups
table = table.loc[table.index.value_counts() >= 3]

table

# NOTE: To see all rows, wrap the display command like this:
#     with pd.option_context('display.max_rows', None):
#         display(table)

comment  \
gender age          postcode job                                                                    
Female [40.0, 50.0) 75xxx    นักสังคมสงเคราะห์  ย้ำรอทอดมันประสบการณ์อิฐเรียงของหวาน หล่นโรงสี...   
                             นักสังคมสงเคราะห์  ประชาธิปไตยอาหารชะนีนักเรียนธาตุแคะ ทวิตเตอร์พ...   
                             นักสังคมสงเคราะห์  เป็นตู้หึงสา ถีบถึงเคเอฟซีหมอกเนื่องจากแม้ \nค...   
       [60.0, inf)  21xxx    นักประวัติศาสตร์   ตั้งแต่ย่อกิจกรรมวิชาชีพบูชา สำนักงานแคะการเผื...   
                             นักประวัติศาสตร์   ห่วงใยตรวจคนตายริม ร้อนแท็กซี่เอสซีบีบันไดแหย่...   
...                                                                                           ...   
Male   [60.0, inf)  88xxx    นักโบราณคดี        ตัวอย่างค่อยที่ดินตัวอย่างมะเขือเทศคำถาม กิจกร...   
                             นักโบราณคดี        นักเรียนสีเทาแต่งสุขภาพเขยื้อนศาลากลางแสงโง่ ย...   
                    95xxx    นักการทูต          บูชาโตหน้าต่างบริโภคคู่การ ทรัพย์สินกล่องแถบคม...   
                             นักการทูต          แผนที่ลืมคาวทุ่มแผนที่ค่อย ไชโยทวิตเตอร์หมีเกา...   
                             นักการทูต          ชนิดชนิดศิษย์ของหวานต่อฝากแคะ กำทุกข์แก้ทอดมัน...   

                                                spending  
gender age          postcode job                          
Female [40.0, 50.0) 75xxx    นักสังคมสงเคราะห์     12000  
                             นักสังคมสงเคราะห์     11000  
                             นักสังคมสงเคราะห์     10000  
       [60.0, inf)  21xxx    นักประวัติศาสตร์      10000  
                             นักประวัติศาสตร์      11000  
...                                                  ...  
Male   [60.0, inf)  88xxx    นักโบราณคดี           13000  
                             นักโบราณคดี           13000  
                    95xxx    นักการทูต             11000  
                             นักการทูต             12000  
                             นักการทูต             12000  

[125 rows x 2 columns]

### เทคนิค anonymization ที่ถูกใช้ ได้แก่

- กำจัดข้อมูลระบุตัวทางตรงออก (**de-identification**) ด้วยการตัดข้อมูลประเภทนั้นออก (**attribute suppression**)
- บอกกลุ่มอายุแทนวันเกิดตรงๆ (**bucketing**)
- mask เลข postcode บางส่วน (**masking**)
- ปัดเศษตัวเลขการใช้จ่ายให้เป็นจำนวนเต็มพันที่ใกล้ที่สุด (**rounding**)
- ตัดรายการที่มีข้อมูลโดดเด่นไม่เข้าพวกออก (**record suppression**)

### ข้อสังเกต

**สมมุติว่า** มีบุคคลภายนอกคนหนึ่งที่ได้รับข้อมูลนิรนามนี้ไป บังเอิญว่าบุคคลผู้นี้ทำงานเป็นนักสังคมสงเคราะห์ จึงมีเข้ามูลเพิ่มเติมจากความรู้ส่วนตัวว่า "นักสังคมสงเคราะห์หญิงอายุ 40 ปีที่เข้าร่วมโครงนี้" มีชื่อว่า A, B, และ C

**ถามว่า** บุคคลผู้นี้จะสามารถเชื่อมโยงข้อมูลทั้งสองเข้าด้วยกัน เพื่อระบุว่า A, B, และ C แต่ละคนเบิกค่าใช้จ่ายละ comment ว่าอะไรได้หรือไม่

**คำตอบ**

- บุคคลนี้ทราบไปว่า A, B และ C อยู่ในกลุ่มที่เบิกค่าใช้จ่ายไปในช่วง 10000 ถึง 12000 และในกลุ่มมี comment อะไรบ้าง
- แต่ไม่สามารถจะจับคู่ค่าใช้จ่ายและcomment เข้ากับชื่อในลักษณะ 1-ต่อ-1 ได้ เนื่องจากทั้งสามรายการมีข้อมูลสาธารณะที่อาจระบุตัวตนได้ (indirect identifier) เหมือนกันทั้งสามคน

นั่นคือ ผู้ได้รับข้อมูลท่านนี้ ไม่สามารถจะระบุตัวตนให้แคบกว่ากลุ่มขนาด 3 คนได้

## Anonymized Dataset as Summary Statistics

เช่น ให้ค่า summary statistics ตามการจัดกลุ่มแบบต่างๆ ดังนี้

In [4]:
stat = original.copy()
stat['age'] = datetime.now().year - stat['date_of_birth'].dt.year

In [5]:
stat[['gender', 'age', 'spending']].groupby('gender').describe()

age                                                     spending  \
        count       mean        std  min   25%   50%   75%    max    count   
gender                                                                       
Female  486.0  54.286008  33.072593  0.0  26.0  52.0  84.0  116.0    486.0   
Male    514.0  57.476654  32.663033  0.0  29.0  58.0  86.0  116.0    514.0   

                                                                        \
                mean          std     min       25%      50%       75%   
gender                                                                   
Female   9979.236626  1038.266537  7373.0   9304.25  10012.5  10630.00   
Male    11976.114786  1008.753441  8388.0  11357.50  12005.5  12605.25   

                 
            max  
gender           
Female  12914.0  
Male    14946.0

In [6]:
stat[['postcode', 'age', 'spending']].groupby('postcode').describe()

age                                                        \
          count       mean        std  min    25%   50%    75%    max   
postcode                                                                
21410      91.0  52.934066  34.682625  0.0  25.00  45.0  86.00  115.0   
22730     108.0  60.703704  33.439654  2.0  33.50  60.0  89.75  116.0   
41560      99.0  52.767677  33.186802  2.0  24.00  46.0  83.50  115.0   
57110     114.0  56.184211  32.457615  0.0  24.25  55.5  84.75  115.0   
66330      95.0  56.947368  33.347101  2.0  26.00  58.0  87.00  115.0   
75200      99.0  57.050505  32.972275  1.0  27.50  59.0  84.50  115.0   
75320     101.0  56.762376  35.520458  0.0  27.00  58.0  87.00  116.0   
88530      98.0  58.673469  30.915173  1.0  32.00  57.0  83.75  115.0   
95840     112.0  51.660714  31.152214  1.0  25.75  49.5  77.25  114.0   
98860      83.0  55.385542  31.314623  1.0  27.00  51.0  79.50  110.0   

         spending                                                        \
            count          mean          std     min       25%      50%   
postcode                                                                  
21410        91.0  10807.252747  1390.366519  7651.0   9681.00  10943.0   
22730       108.0  11007.638889  1498.449338  7920.0  10063.50  11032.5   
41560        99.0  11201.585859  1452.424820  7981.0  10254.00  11399.0   
57110       114.0  11070.578947  1391.809403  7907.0  10167.75  10881.5   
66330        95.0  10974.136842  1442.140472  7570.0   9963.50  11043.0   
75200        99.0  11262.878788  1411.795197  8216.0  10144.50  11466.0   
75320       101.0  10734.029703  1232.670162  7835.0   9794.00  10760.0   
88530        98.0  11122.010204  1508.917817  7373.0  10052.25  11244.5   
95840       112.0  11008.991071  1521.419273  7681.0   9880.25  10811.0   
98860        83.0  10815.361446  1371.433891  7618.0   9891.50  10971.0   

                             
               75%      max  
postcode                     
21410     11889.00  13594.0  
22730     12076.00  14911.0  
41560     12169.00  14381.0  
57110     12145.50  14266.0  
66330     12084.00  14033.0  
75200     12481.00  13762.0  
75320     11640.00  13712.0  
88530     12167.25  14946.0  
95840     12291.25  14309.0  
98860     11834.00  13470.0

In [7]:
stat[['job', 'spending']].groupby('job').describe()

spending                                               \
                        count          mean          std     min       25%   
job                                                                          
คนขับรถแท็กซี่           39.0  11005.743590  1459.235050  7907.0   9944.00   
จ๊อกกี้                  31.0  11109.419355  1353.431682  7977.0  10185.50   
ช่างทำผม                 43.0  11193.372093  1502.571241  7923.0   9994.50   
ตำรวจ                    43.0  10694.651163  1348.288613  8512.0   9727.00   
นักการกุศล               35.0  10910.542857  1520.707698  7758.0   9983.50   
นักการทูต                37.0  10619.594595  1625.218164  7934.0   9360.00   
นักการศึกษา              34.0  11197.882353  1336.344693  8251.0  10533.75   
นักคณิตศาสตร์            25.0  10835.800000  1374.248401  8196.0   9935.00   
นักดาราศาสตร์            29.0  11331.896552  1612.909491  7373.0  10350.00   
นักประวัติศาสตร์         31.0  10845.935484  1493.878441  7651.0   9923.50   
นักพจนานุกรม             29.0  10973.689655  1460.722035  8448.0   9850.00   
นักมายากล                28.0  10899.142857  1495.788575  8132.0  10049.50   
นักสังคมศาสตร์           39.0  10498.205128  1281.778250  7767.0   9864.00   
นักสังคมสงเคราะห์        32.0  10930.062500  1430.698587  7920.0  10021.25   
นักสัตววิทยา             28.0  11150.071429  1468.800290  8712.0  10046.75   
นักเขียน                 27.0  11268.000000  1619.759906  7792.0  10276.00   
นักแปล                   39.0  11279.487179  1228.066428  9028.0  10477.00   
นักแสดง                  42.0  10782.000000  1162.804054  8617.0   9613.75   
นักโบราณคดี              28.0  11595.035714  1248.726951  9199.0  10698.25   
นักโภชนาการ              33.0  11164.424242  1460.493817  8222.0   9812.00   
บรรณารักษ์               30.0  11295.133333  1504.696412  7681.0  10569.50   
ผู้กำกับภาพยนตร์         36.0  10635.027778  1397.008630  8047.0   9603.50   
ผู้กำกับละครโทรทัศน์     27.0  11367.037037  1290.908669  8257.0  10681.50   
ผู้จัดพิมพ์              40.0  11228.775000  1409.441670  7835.0  10311.75   
ภัณฑารักษ์               33.0  10685.636364  1531.620788  8509.0   9688.00   
ศิลปิน                   38.0  10858.526316  1162.051256  8435.0  10030.75   
อาจารย์                  25.0  11679.760000  1372.395633  9297.0  10762.00   
เกษตรกร                  35.0  10660.114286  1635.593898  7570.0   9290.00   
เจ้าหน้าที่รัฐบาล        28.0  10865.535714  1384.300237  8462.0   9944.50   
แพทย์                    36.0  11175.888889  1436.562022  7928.0  10253.25   

                                                  
                          50%       75%      max  
job                                               
คนขับรถแท็กซี่        11042.0  12086.00  13804.0  
จ๊อกกี้               11472.0  12069.00  13454.0  
ช่างทำผม              11218.0  12341.00  14911.0  
ตำรวจ                 10519.0  11612.00  13736.0  
นักการกุศล            10910.0  11949.00  13207.0  
นักการทูต             10440.0  11905.00  13498.0  
นักการศึกษา           11193.0  11876.00  13712.0  
นักคณิตศาสตร์         10542.0  11939.00  13993.0  
นักดาราศาสตร์         11372.0  12600.00  14381.0  
นักประวัติศาสตร์      10534.0  11903.50  13500.0  
นักพจนานุกรม          11200.0  12195.00  14309.0  
นักมายากล             10818.0  12296.00  13326.0  
นักสังคมศาสตร์        10384.0  11302.50  13259.0  
นักสังคมสงเคราะห์     11026.5  12075.25  13163.0  
นักสัตววิทยา          11069.0  12162.25  13835.0  
นักเขียน              11814.0  12512.50  13743.0  
นักแปล                11240.0  12144.00  14033.0  
นักแสดง               10586.5  11826.25  12635.0  
นักโบราณคดี           12025.0  12552.25  13088.0  
นักโภชนาการ           11436.0  12292.00  13367.0  
บรรณารักษ์            11599.0  12159.25  14266.0  
ผู้กำกับภาพยนตร์      10810.0  11246.50  14946.0  
ผู้กำกับละครโทรทัศน์  11523.0  12227.50  13802.0  
ผู้จัดพิมพ์           11129.5  12243.75  13778.0  
ภัณฑารักษ์            10305.0  11300.00  14192.0  
ศิลปิน            

### บรรยายแค่สถิติ ก็ยังระบุตัวตนได้

- ถ้าประกาศสิถิติหลายๆรูปแบบจากข้อมูลเดียวกัน ก็สามารถที่จะนำสถิติแต่ละตัวมาตั้งสมการ เพื่อแก้หาค่าตั้งต้นที่อยู่ในชุดข้อมูลที่ไม่ได้ถูกเปิดเผยโดยตรงได้
- ตัวอย่างเช่น
  - 
- การโจมตีเช่นนี้เรียกว่า
  - [Database Reconstruction Attack](https://dl.acm.org/doi/pdf/10.1145/3287287)
  - Garfinkel, Simson, John M. Abowd, and Christian Martindale. "Understanding database reconstruction attacks on public data." Communications of the ACM 62.3 (2019): 46-53.

- เช่น ค่าเฉลี่ยแยกตามอาชีพ, ค่าเฉลี่ยแยกตามที่อยู่
- เพราะถ้าประกาศสิถิติกลายรูปแบบที่บรรยายชุดข้อมูลเดียวกันเป็นจำนวณมากพอ ก็สามารถที่จะตั้งตัวแปรเป็นค่าข้อมูลของแต่ละคน แล้วใช้สถิติที่ได้บรรยายเงื่อนไขสมการณ์หลายตัวแปร แก้ดึงตัวแปรกลับมาได้

## Anonymized Dataset as API

- ใส่ noise ก่อนจะตอบได้
- วัดระดับความนิรนามด้วย Differential Privacy (ถ้าข้อมูลเป็นตัวเลขทั้งหมด)
- ต้องตั้ง privacy budget ไว้ ซึ่งจะถูกลดทอนไปเรื่อยๆเมื่อข้อมูลถูก export
- เมื่อ privacy budget หมด ต้องหยุดให้บริการ
  - เพราะมีการบอกข้อมูลกับโลกภายนอกมากเกินไปแล้ว
  - ถึงจะไม่ได้บอกค่าตรงๆ แต่ถ้าบอกเป็นจำนวนครั้งมากพอก็นำมารวมสถิคิได้
- ควบคุมจำนวนครั้งการให้บริการได้

### Differential Privacy

![](https://upload.wikimedia.org/wikipedia/commons/d/db/Differential_privacy_informal_definition.png)

![](https://upload.wikimedia.org/wikipedia/commons/3/3a/Differential_privacy_formal_definition.png)

#### Benefits of Differential Privacy

Differential privacy has several important advantages over previous privacy techniques:

- It **assumes all information is identifying information**, eliminating the challenging (and sometimes impossible) task of accounting for all identifying elements of the data.
- It is **resistant to privacy attacks based on auxiliary information**, so it can effectively prevent the linking attacks that are possible on de-identified data.
- It is **compositional** — we can determine the privacy loss of running two differentially private analyses on the same data by simply adding up the individual privacy losses for the two analyses. Compositionality means that we can make meaningful guarantees about privacy even when releasing multiple analysis results from the same data.  Techniques like de-identification are not compositional, and multiple releases under these techniques can result in a catastrophic loss of privacy.

https://www.nist.gov/blogs/cybersecurity-insights/differential-privacy-privacy-preserving-data-analysis-introduction-our

# Next Steps

- [ฺBasic Anonymization Techniques](../techniques/basic.ipynb) แต่ละวิธี
- การวัดระดับความนิรนามโดย [K-anonymity](../k_anonymity/k_anonymity.ipynb), L-diversity, และ T-closeness
- [Differential Privacy](../diff/differential_privacy.ipynb)